# Homework 11 

- INFO 5871-001
- Keke Wu
- Fall 2019

## Imports

Gensim and pyLDAvis are _not_ standard packages with Anaconda. You will need to uncomment and run the following cell to install them.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/keke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/keke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/keke/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Aim

The aim of this homework is to introduce LDA for topic modeling using the Trump false statements database. We load the CSV file as before.

In [3]:
trump = pd.read_csv('wapo_trumpclaims_export-080519.csv')
print (trump.shape)
trump.head()

(12020, 8)


,id,date,source,topic,claim,analysis,repeated_count,repeated_ids
0,11702,08/05/19,Twitter,Guns,“The Media has a big responsibility to life an...,"There's <a href=""https://psmag.com/news/does-t...",0.0,NaN
1,11703,08/05/19,Twitter,Trade,“China dropped the price of their currency to ...,China's currency dropped to an 11-year low aga...,0.0,NaN
2,11704,08/05/19,Twitter,Trade,“Based on the historic currency manipulation b...,"We calculated that through December, the Trump...",73.0,"11704, 11700, 11166, 11307, 11575, 11629, 1135..."
3,11706,08/05/19,Twitter,Trade,“China is intent on continuing to receive the ...,Trump consistently fails to understand trade d...,166.0,"11706, 11576, 11388, 12211, 11552, 11394, 1218..."
4,12154,08/05/19,Prepared speech,Guns,"“Mental illness and hatred pulls the trigger, ...","The Washington Post <a href=""https://www.washi...",0.0,NaN


## The corpus

Create a corpus by joining together the entries in the `claim` and `analysis` columns. With just the claims, there isn't quite enough text for LDA to work well. Note that these preprocessing steps will be similar to those in Homework 10, but instead of operating on the whole set of claims, we want to operate on a list of entries/documents. 

In [4]:
trump['corpus'] = trump['claim'] + trump['analysis']
datalst = list(trump.corpus)
data = [str(item) for item in datalst]
data[0]

'“The Media has a big responsibility to life and safety in our Country. Fake News has contributed greatly to the anger and rage that has built up over many years. News coverage has got to start being fair, balanced and unbiased, or these terrible problems will only get worse!”There\'s <a href="https://psmag.com/news/does-the-media-cause-mass-shootings">some</a> evidence to suggest that intense media coverage of a shooting could be partially to blame for a future mass shooting, but extensive coverage isn\'t the same as not being "fair" or "balanced," as Trump suggests. Plus, it\'s not likely journalist will just stop covering mass shootings, in part, as Pacific Standard explained, "media organizations have an ethical responsibility to bear witness to the tragedies of history, to ensure that future generations don’t forget the horrors of epochs past."'

### Tokenize the entries and remove punctuation

In [5]:
data_tokens = [word_tokenize(entry) for entry in data]
data_words  = [[word.lower() for word in entry if word.isalpha()] for entry in data_tokens]

## Remove stopwords

In [6]:
english_stops = set(stopwords.words('english'))
data_stops = [[word for word in entry if word not in english_stops] for entry in data_words]

### Lemmatize

In [7]:
lemmatizer = WordNetLemmatizer()
data_lem = [[lemmatizer.lemmatize(word) for word in entry] for entry in data_stops]
print(data_lem[0])
print(len(data_lem))

['medium', 'big', 'responsibility', 'life', 'safety', 'country', 'fake', 'news', 'contributed', 'greatly', 'anger', 'rage', 'built', 'many', 'year', 'news', 'coverage', 'got', 'start', 'fair', 'balanced', 'unbiased', 'terrible', 'problem', 'get', 'worse', 'http', 'evidence', 'suggest', 'intense', 'medium', 'coverage', 'shooting', 'could', 'partially', 'blame', 'future', 'mass', 'shooting', 'extensive', 'coverage', 'fair', 'balanced', 'trump', 'suggests', 'plus', 'likely', 'journalist', 'stop', 'covering', 'mass', 'shooting', 'part', 'pacific', 'standard', 'explained', 'medium', 'organization', 'ethical', 'responsibility', 'bear', 'witness', 'tragedy', 'history', 'ensure', 'future', 'generation', 'forget', 'horror', 'epoch', 'past']
12020


### Hint

At this point, you should have a list of 12,020 lists. The first item in the list should look something like this:

```['medium', 'big', 'responsibility', 'life', 'safety', 'country', 'fake', 'news', 'contributed', 'greatly', 'anger', 'rage', 'built', 'many', 'year', 'news', 'coverage', 'got', 'start', 'fair', 'balanced', 'unbiased', 'terrible', 'problem', 'get', 'worse', 'http', 'evidence', 'suggest', 'intense', 'medium', 'coverage', 'shooting', 'could', 'partially', 'blame', 'future', 'mass', 'shooting', 'extensive', 'coverage', 'fair', 'balanced', 'trump', 'suggests', 'plus', 'likely', 'journalist', 'stop', 'covering', 'mass', 'shooting', 'part', 'pacific', 'standard', 'explained', 'medium', 'organization', 'ethical', 'responsibility', 'bear', 'witness', 'tragedy', 'history', 'ensure', 'future', 'generation', 'forget', 'horror', 'epoch', 'past']```


### Create an `id2word` dictionary

In [8]:
id2word = gensim.corpora.Dictionary(data_lem)

### Create a bag-of-words representation for each entry

In [9]:
data_id = [id2word.doc2bow(entry) for entry in data_lem]
print(data_id[0])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 3), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 3), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 3), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)]


### Count how many manually-assigned topics there are 

In [10]:
len(pd.value_counts(trump.topic))

17

## Create LDA model

Use the number of topics that the Washington Post assigned. There's no guarantee that LDA will match them exactly. Use the same settings as we did in class.

This takes some minutes. Some library versions issue a warning on every LDA iteration with is really annoying.

In [11]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=data_id, id2word=id2word, num_topics=17,
                                              random_state=20191119, update_every=1, chunksize=100,
                                              passes=10, alpha='auto', per_word_topics=True)

### Print the topics

In [12]:
lda_model.print_topics()

[(0,
  '0.056*"white" + 0.051*"defense" + 0.050*"city" + 0.042*"call" + 0.038*"obama" + 0.038*"rule" + 0.037*"reduction" + 0.034*"remain" + 0.029*"developed" + 0.027*"military"'),
 (1,
  '0.096*"increase" + 0.031*"government" + 0.028*"lowest" + 0.025*"veteran" + 0.024*"issued" + 0.024*"step" + 0.022*"clear" + 0.020*"va" + 0.019*"historic" + 0.018*"recession"'),
 (2,
  '0.117*"trump" + 0.036*"people" + 0.027*"said" + 0.022*"president" + 0.021*"time" + 0.020*"one" + 0.018*"say" + 0.017*"many" + 0.016*"year" + 0.015*"new"'),
 (3,
  '0.063*"vote" + 0.053*"mexico" + 0.048*"obamacare" + 0.041*"figure" + 0.027*"order" + 0.024*"small" + 0.023*"among" + 0.022*"false" + 0.021*"percentage" + 0.020*"research"'),
 (4,
  '0.098*"state" + 0.075*"country" + 0.063*"united" + 0.046*"going" + 0.025*"deal" + 0.024*"trillion" + 0.022*"work" + 0.019*"also" + 0.019*"worker" + 0.018*"world"'),
 (5,
  '0.111*"rate" + 0.102*"percent" + 0.079*"american" + 0.051*"unemployment" + 0.033*"average" + 0.028*"statistic

### Print `log_perplexity`

In [13]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    print(lda_model.log_perplexity(data_id))

-11.318696517962755


### Compute the model coherence

In [14]:
coherence_model = CoherenceModel(model=lda_model, texts=data_lem, dictionary=id2word, coherence='c_v')
coherence_model.get_coherence()

0.44334501665275566

## Create an LDAvis for the model

This also takes some time.

In [15]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, data_id, id2word)
vis

//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.246636 -0.081823       1        1  22.305573
4     -0.047178 -0.111838       2        1   9.954291
14    -0.244832  0.134948       3        1   8.666994
10    -0.102452  0.003869       4        1   8.662300
12     0.119110 -0.076184       5        1   8.096353
9     -0.033479 -0.240562       6        1   7.639371
11    -0.107899 -0.162447       7        1   5.033378
5     -0.141613  0.264086       8        1   5.004584
7      0.041481 -0.034431       9        1   4.480750
3      0.050106 -0.057864      10        1   3.963299
16     0.136439 -0.053858      11        1   3.527309
1      0.007404  0.053465      12        1   2.747051
0      0.094393 -0.055208      13        1   2.397985
8      0.050430  0.074266      14        1   2.030525
13     0.064111  0.174867      15        1   2.007684
15     0.115716  0.084478      16        1   1.965970
6      0.244897  0.084238      17        1   1.516583, topic_info=     Category          Freq         Term         Total  loglift  logprob
430   Default   6169.000000        state   6169.000000  30.0000  30.0000
284   Default   6211.000000      percent   6211.000000  29.0000  29.0000
136   Default   4719.000000          tax   4719.000000  28.0000  28.0000
453   Default   4640.000000          job   4640.000000  27.0000  27.0000
166   Default   3430.000000         rate   3430.000000  26.0000  26.0000
8     Default   5336.000000      country   5336.000000  25.0000  25.0000
54    Default  21274.000000        trump  21274.000000  24.0000  24.0000
437   Default   3945.000000       united   3945.000000  23.0000  23.0000
90    Default   3585.000000      company   3585.000000  22.0000  22.0000
316   Default   4490.000000        would   4490.000000  21.0000  21.0000
84    Default   4035.000000      billion   4035.000000  20.0000  20.0000
803   Default   3925.000000        obama   3925.000000  19.0000  19.0000
79    Default   3273.000000     american   3273.000000  18.0000  18.0000
1350  Default   2102.000000         wall   2102.000000  17.0000  17.0000
491   Default   1739.000000         plan   1739.000000  16.0000  16.0000
267   Default   2873.000000        going   2873.000000  15.0000  15.0000
229   Default   1753.000000     increase   1753.000000  14.0000  14.0000
330   Default   2585.000000       russia   2585.000000  13.0000  13.0000
115   Default   3799.000000      million   3799.000000  12.0000  12.0000
172   Default   2927.000000        trade   2927.000000  11.0000  11.0000
164   Default   5262.000000       people   5262.000000  10.0000  10.0000
348   Default   2249.000000      clinton   2249.000000   9.0000   9.0000
734   Default   1399.000000       market   1399.000000   8.0000   8.0000
127   Default   2210.000000      product   2210.000000   7.0000   7.0000
96    Default   2312.000000          cut   2312.000000   6.0000   6.0000
1080  Default   1528.000000        crowd   1528.000000   5.0000   5.0000
765   Default   2224.000000       border   2224.000000   4.0000   4.0000
502   Default   1553.000000         vote   1553.000000   3.0000   3.0000
83    Default   2115.000000         bill   2115.000000   2.0000   2.0000
345   Default   2387.000000     campaign   2387.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
759   Topic17    447.552979        woman    449.086334   4.1853  -3.0421
554   Topic17    304.103424       police    305.636780   4.1837  -3.4286
1859  Topic17    252.173157       needed    253.706512   4.1826  -3.6158
534   Topic17    230.043091         hold    231.576447   4.1821  -3.7077
542   Topic17    201.558853          kim    203.092209   4.1811  -3.8398
1861  Topic17    169.026855     progress    170.560211   4.1797  -4.0159
1855  Topic17    164.407242      covered    165.940598   4.1794  -4.0436
1857  Topic17    145.839417        final    147.372772   4.1783  -4.1634
930   Topic

### Explore the topics with the visualization. 

Which topic(s) seem to correspond to the topic of immigration?

_5, 8, 9, 10, 11_

### Get topics for the term `immigration`

In [16]:
lda_model.get_term_topics('immigration')

[(12, 0.031536512)]

### Get features for the topic most identified with this term

In [17]:
lda_model.show_topic(12)

[('democrat', 0.04005868),
 ('fbi', 0.03275987),
 ('republican', 0.031649545),
 ('immigration', 0.031551417),
 ('border', 0.029264875),
 ('want', 0.024678482),
 ('illegal', 0.023210555),
 ('law', 0.021639742),
 ('overseas', 0.019829031),
 ('support', 0.019467495)]

### Create a model with 25 topics

In [18]:
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=data_id, id2word=id2word, num_topics=25,
                                              random_state=20191119, update_every=1, chunksize=100,
                                              passes=10, alpha='auto', per_word_topics=True)

### Compare coherence

In my version, this is lower (worse) than the 17 topic version. YMMV.

In [19]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    print(lda_model2.log_perplexity(data_id))

-13.132960198604337


### Create a "long" data frame with the columns doc_no, topic, weight

Use whichever model you want. You'll need a for loop.

In [75]:
document_topics = [lda_model.get_document_topics(item) for item in data_id]
document_topics[0:2]
# 0, 2, 3.. are the topics, each of them is followed by the weight

[[(0, 0.02273394),
  (2, 0.67751193),
  (3, 0.022355443),
  (4, 0.0769556),
  (5, 0.03237389),
  (9, 0.0208904),
  (10, 0.043760724),
  (12, 0.020907074),
  (13, 0.02157689),
  (14, 0.017304271),
  (16, 0.018660694)],
 [(0, 0.042132523),
  (1, 0.06839298),
  (2, 0.19772606),
  (3, 0.012168645),
  (4, 0.054363515),
  (5, 0.111401044),
  (6, 0.055969197),
  (7, 0.01279043),
  (10, 0.25151044),
  (11, 0.118115485),
  (12, 0.016968668),
  (14, 0.028460441)]]

In [94]:
long_columns = {"topic":trump.topic,
             "doc_no": trump.id,
             "weight": get_document_topics}
long = pd.DataFrame(long_columns)
long.head()

,topic,doc_no,weight
0,Guns,11702,"[(0, 0.022749478), (2, 0.6775118), (3, 0.02235..."
1,Trade,11703,"[(0, 0.042158343), (1, 0.06839272), (2, 0.1977..."
2,Trade,11704,"[(1, 0.026210455), (2, 0.065457195), (4, 0.010..."
3,Trade,11706,"[(0, 0.01888565), (2, 0.094616346), (4, 0.3819..."
4,Guns,12154,"[(1, 0.03402878), (2, 0.2738663), (3, 0.064743..."


### Use the data frame to find the sentence with highest weight for the `immigration` topic 

In [88]:
immigration_topic = long[long['topic']=='Immigration']
# immigration_topic.weight

Print the associated entry from the corpus (claim and analysis)